Install dependencies

In [ ]:
!pip install kaggle torch torchvision

import os
import zipfile
import torch
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import sklearn.metrics as metrics
import json

Kaggle Setup

In [ ]:
#This step requires your own kaggle.json file.
# Get it from your kaggle account (Create New API Token).
# Then upload it when prompted.

from google.colab import files
print("upload your kaggle.json (from Kaggle API)")
files.upload()

# Create the .kaggle directory
os.makedirs("/root/.kaggle", exist_ok=True)

# Move kaggle.json to the .kaggle directory
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


Download Dataset

In [ ]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

# Unzip dataset
with zipfile.ZipFile("new-plant-diseases-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("plant_disease_dataset")

base_dir = "plant_disease_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"
print("Dataset contents:", os.listdir(base_dir))

Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
100% 2.69G/2.70G [00:26<00:00, 61.6MB/s]
100% 2.70G/2.70G [00:26<00:00, 108MB/s] 
Dataset contents: ['train', 'valid']


Set Paths

In [ ]:
train_path = os.path.join(base_dir, "train")
val_path = os.path.join(base_dir, "valid")

 Define transforms

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

Load datasets

In [ ]:
train_data = datasets.ImageFolder(train_path, transform=transform)
val_data = datasets.ImageFolder(val_path, transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)

 Load Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v3_small(pretrained=True)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, len(train_data.classes))
model = model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100%|██████████| 9.83M/9.83M [00:00<00:00, 42.7MB/s]


Loss & Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Training Loop

In [ ]:
for epoch in range(15):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1, Loss: 0.3055
Epoch 2, Loss: 0.0372
Epoch 3, Loss: 0.0212
Epoch 4, Loss: 0.0148
Epoch 5, Loss: 0.0107
Epoch 6, Loss: 0.0099
Epoch 7, Loss: 0.0082
Epoch 8, Loss: 0.0070
Epoch 9, Loss: 0.0062
Epoch 10, Loss: 0.0057
Epoch 11, Loss: 0.0050
Epoch 12, Loss: 0.0053
Epoch 13, Loss: 0.0041
Epoch 14, Loss: 0.0049
Epoch 15, Loss: 0.0041


Validation

In [ ]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

accuracy = metrics.accuracy_score(y_true, y_pred)
print(f"✅ Validation Accuracy: {accuracy:.4f}")

✅ Validation Accuracy: 0.9975


Save Model

In [ ]:
torch.save(model.state_dict(), "mobilenetv3_plant_disease.pth")
with open("class_labels.json", "w") as f:
    json.dump(train_data.class_to_idx, f)

print("Model and class labels saved successfully.")

Model and class labels saved successfully.


In [3]:
import torch
from torchvision import models, transforms
from PIL import Image
import json

# Load class labels
with open("../models/class_labels.json", "r") as f:
    class_to_idx = json.load(f)
idx_to_class = {v: k for k, v in class_to_idx.items()}

# Load model
num_classes = len(class_to_idx)
model = models.mobilenet_v3_small(pretrained=False)
model.classifier[3] = torch.nn.Linear(model.classifier[3].in_features, 38)
model.load_state_dict(torch.load("../models/mobilenetv3_plant_disease.pth", map_location='cpu'))
model.eval()

# Define transform (same as training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Prediction function
def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    input_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_tensor)
        _, pred = torch.max(outputs, 1)
        class_name = idx_to_class[pred.item()]
    return class_name

# Example usage:
# result = predict_image("test_leaf.jpg")
# print("Predicted disease:", result)

In [5]:
def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    input_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_tensor)
        _, pred = torch.max(outputs, 1)
        class_idx = pred.item()
        class_name = idx_to_class.get(class_idx, f"Class {class_idx}")
    return class_name

result = predict_image("plant-disease.jpeg")
print("Predicted disease:", result)

Predicted disease: Class 0


In [6]:
print(idx_to_class[0])

KeyError: 0